In [ ]:
import pandas as pd
import numpy as np

import copy
import pickle
from datetime import datetime

import xgboost as xgb
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import median_absolute_error as MAPE
from sklearn.metrics import r2_score  as R2
from xgboost import plot_importance
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

# Parameter tunning
import optuna
from sklearn.model_selection import KFold

import shap

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

Data description:
- 1127 postcodes
- Google POI + EPC statistics + Property Data

Goal: sqf prices

https://machinelearningmastery.com/xgboost-for-regression/
https://xgboost.readthedocs.io/en/stable/parameter.html


In [ ]:
def date_to_year(date_string):
    fmt = '%Y-%m-%d'
    return datetime.strptime(date_string,fmt).year

In [ ]:
Data = pd.read_csv('datasetV6.csv') 

In [ ]:
len(Data)

In [ ]:
list(Data.columns)

In [ ]:
Data['synagogue_exist'] = Data['synagogue_count_500']>0
Data['shopping_exist'] = Data['shopping_mall_count_500']>0
Data['secondary_exist'] = Data['secondary_count_500']>0
Data['school_exist'] = Data['school_count_500']>0
Data['primary_exist'] = Data['primary_count_500']>0
Data['police_exist'] = Data['police_count_500']>0
Data['parking_exist'] = Data['parking_count_500']>0
Data['park_exist'] = Data['park_count_500']>0
Data['mosque_exist'] = Data['mosque_count_500']>0
Data['fire_station_exist'] = Data['fire_station_count_500']>0
Data['hindu_temple_exist'] = Data['hindu_temple_count_500']>0
Data['cemetery_exist'] = Data['cemetery_count_500']>0
Data['university_exist'] = Data['university_count_500']>0
Data['train_station_exist'] = Data['train_station_count_500']>0
Data['hospital_exist'] = Data['hospital_count_500']>0
Data['cafe_exist'] = Data['cafe_count_500']>0
Data['supermarket_exist'] = Data['supermarket_count_500']>0
Data['subway_station_exist'] = Data['subway_station_count_500']>0
Data['bars_exist'] = Data['bars_count_500']>0
Data['restaurants_exist'] = Data['restaurants_count_500']>0

### Crime

In [ ]:
Data['crime_total']=Data['PublicOrder']+Data['Burglary']+Data['Robbery']+Data['PossessionOfWeapons']+Data['BicycleTheft']+Data['AntiSocialBehaviour']+Data['Violence']+Data['Theft']+Data['Shoplifting']+Data['OtherCrime']+Data['Drugs']+Data['OtherTheft']+Data['VehicleCrime']+Data['CriminalDamage']

In [ ]:
Data['PublicOrder']=Data['PublicOrder']/Data['crime_total']

In [ ]:
Data['Burglary'] = Data['Burglary']/Data['crime_total']
Data['Robbery'] = Data['Robbery']/Data['crime_total']
Data['PossessionOfWeapons'] = Data['PossessionOfWeapons']/Data['crime_total']
Data['BicycleTheft'] = Data['BicycleTheft']/Data['crime_total']
Data['AntiSocialBehaviour'] = Data['AntiSocialBehaviour']/Data['crime_total']
Data['Violence'] = Data['Violence']/Data['crime_total']
Data['Theft'] = Data['Theft']/Data['crime_total']
Data['Shoplifting'] = Data['Shoplifting']/Data['crime_total']
Data['OtherCrime'] = Data['OtherCrime']/Data['crime_total']
Data['Drugs'] = Data['Drugs']/Data['crime_total']
Data['OtherTheft'] = Data['OtherTheft']/Data['crime_total']
Data['VehicleCrime'] = Data['VehicleCrime']/Data['crime_total']
Data['CriminalDamage'] = Data['CriminalDamage']/Data['crime_total']

Data['crime_Type_A'] = Data['Robbery']+Data['Drugs']+Data['PossessionOfWeapons']+Data['Burglary']+Data['OtherCrime']
Data['crime_Type_B'] = Data['OtherTheft'] + Data['Shoplifting'] + Data['Theft'] + Data['BicycleTheft']
Data['crime_Type_C'] = Data['CriminalDamage']+ Data['Violence'] + Data['AntiSocialBehaviour']
Data['crime_Type_D'] = Data['VehicleCrime']

In [ ]:
dropList=['Burglary',
 'Robbery',
 'PossessionOfWeapons',
 'BicycleTheft',
 'AntiSocialBehaviour',
 'Violence',
 'Theft',
 'Shoplifting',
 'OtherCrime',
 'Drugs',
 'OtherTheft',
 'VehicleCrime',
 'CriminalDamage',]
Data.drop(dropList, axis = 1, inplace = True)

### Age 


In [ ]:
Data['TotalModPop'] = Data['age20_24']+Data['age25_29']+Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age45_49']+Data['age50_54']+Data['age60_64']
Data['age20_29'] = Data['age20_24']+Data['age25_29']
Data['age30_64'] = Data['age30_34']+Data['age35_39']+Data['age40_44']+Data['age45_49']+Data['age50_54']+Data['age60_64']
Data['age65_'] = Data['age70_74']+Data['age75_79']+Data['age80_84']+Data['age85_89']

In [ ]:
dropList=['age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',
 'age45_49',
 'age50_54',
 'age60_64',
 'age70_74',
 'age75_79',
 'age80_84',
 'age85_89']
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
Data['commute_method_public'] = Data['commute_method_train'] + Data['commute_method_bus'] + Data['commute_method_underground_light_rail']
Data['commute_method_private'] = Data['commute_method_motorcycle'] + Data['commute_method_taxi'] + Data['commute_method_car_driver'] + Data['commute_method_car_passenger']
Data['commute_method_open'] = Data['commute_method_foot']+Data['commute_method_bicycle']

In [ ]:
dropList=['commute_method_foot',
 'commute_method_bicycle',
 'commute_method_other',
 'commute_method_motorcycle',
 'commute_method_taxi',
 'commute_method_train',
 'commute_method_bus',
 'commute_method_underground_light_rail',
 'commute_method_car_driver',
 'commute_method_at_home',
 'commute_method_car_passenger',
]
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
#Data['sold_70pc_disp']=-(Data['sold_70pc_rangeMIN']-Data['sold_70pc_rangeMAX'])/Data['sold_average']
#Data['sold_80pc_disp']=-(Data['sold_80pc_rangeMIN']-Data['sold_80pc_rangeMAX'])/Data['sold_average']
#Data['sold_90pc_disp']=-(Data['sold_90pc_rangeMIN']-Data['sold_90pc_rangeMAX'])/Data['sold_average']
#Data['sold_100pc_disp']=-(Data['sold_100pc_rangeMIN']-Data['sold_100pc_rangeMAX'])/Data['sold_average']

In [ ]:
#Data['sold_sqf_70pc_disp'] =-(Data['sold_sqf_70pc_rangeMIN']-Data['sold_sqf_70pc_rangeMAX'])/Data['sold_sqf_average']
#Data['sold_sqf_80pc_disp'] =-(Data['sold_sqf_80pc_rangeMIN']-Data['sold_sqf_80pc_rangeMAX'])/Data['sold_sqf_average']
#Data['sold_sqf_90pc_disp'] =-(Data['sold_sqf_90pc_rangeMIN']-Data['sold_sqf_90pc_rangeMAX'])/Data['sold_sqf_average']
#Data['sold_sqf_100pc_disp'] =-(Data['sold_sqf_100pc_rangeMIN']-Data['sold_sqf_100pc_rangeMAX'])/Data['sold_sqf_average']

In [ ]:
#Data['rents_70pc_disp'] = -(Data['rents_70pc_rangeMIN']-Data['rents_70pc_rangeMAX'])/Data['rents_average']
#Data['rents_80pc_disp'] =-(Data['rents_80pc_rangeMIN']-Data['rents_80pc_rangeMAX'])/Data['rents_average']
#Data['rents_90pc_disp'] =-(Data['rents_90pc_rangeMIN']-Data['rents_90pc_rangeMAX'])/Data['rents_average']
#Data['rents_100pc_disp'] =-(Data['rents_100pc_rangeMIN']-Data['rents_100pc_rangeMAX'])/Data['rents_average']

In [ ]:
dropList=[ 'dyn_sold_points_analysed',
 'dyn_sold_70pc_rangeMIN',
 'dyn_sold_70pc_rangeMAX',
 'dyn_sold_80pc_rangeMIN',
 'dyn_sold_80pc_rangeMAX',
 'dyn_sold_90pc_rangeMIN',
 'dyn_sold_90pc_rangeMAX',
 'dyn_sold_sqf_points_analysed',
 'dyn_sold_sqf_70pc_rangeMIN',
 'dyn_sold_sqf_70pc_rangeMAX',
 'dyn_sold_sqf_80pc_rangeMIN',
 'dyn_sold_sqf_80pc_rangeMAX',
 'dyn_sold_sqf_90pc_rangeMIN',
 'dyn_sold_sqf_90pc_rangeMAX',
 'dyn_rents_points_analysed',
 'dyn_rents_70pc_rangeMIN',
 'dyn_rents_70pc_rangeMAX',
 'dyn_rents_80pc_rangeMIN',
 'dyn_rents_80pc_rangeMAX',
 'dyn_rents_90pc_rangeMIN',
 'dyn_rents_90pc_rangeMAX',]
Data.drop(dropList, axis = 1, inplace = True)

## POI

In [ ]:
Data['HperP']=Data['Households']/Data['Population']

In [ ]:
Data['construction_before_1950']=(Data['construction_before_1900'] + Data['construction_1900_1929'] + Data['construction_1930_1949']) 
Data['construction_1950_1975']=(Data['construction_1950_1966']+Data['construction_1967_1975']+ Data['construction_1976_1982'])
Data['construction_1983_1990']=(Data['construction_1983_1990']+Data['construction_1991_1995'])
Data['construction_1996_2006']=(Data['construction_1996_2002']+Data['construction_2003_2006'])
Data['construction_2007_onwards']=(Data['construction_2007_2011']+Data['construction_2012_onwards'])

In [ ]:
dropList=['construction_before_1900',
 'construction_1900_1929',
 'construction_1930_1949',
 'construction_1950_1966',
 'construction_1967_1975',
 'construction_1976_1982',
 'construction_1983_1990',
 'construction_1991_1995',
 'construction_1996_2002',
 'construction_2003_2006',
 'construction_2007_2011',
 'construction_2012_onwards']
Data.drop(dropList, axis = 1, inplace = True)

Removing irrelevant attributes

In [ ]:
list(Data.columns)

In [ ]:
dropList=['Unnamed: 0','postcode','count',
         'postcode_Latitude','postcode_Longitude','MSOA_Code','LSOA_Code','area'
          ]
Data.drop(dropList, axis = 1, inplace = True)

In [ ]:
dropList=['dyn_sold_average',
 'dyn_sold_100pc_rangeMIN',
 'dyn_sold_100pc_rangeMAX',
 'dyn_sold_sqf_100pc_rangeMIN',
 'dyn_sold_sqf_100pc_rangeMAX',
 'dyn_rents_average',
 'dyn_rents_100pc_rangeMIN',
 'dyn_rents_100pc_rangeMAX',
         ]
Data.drop(dropList, axis = 1, inplace = True)

Adding the price paid by sq

### sq_PricePaid statistics

In [ ]:
Data['dyn_sold_sqf_average'].describe()

### Categorical attribute binarization

In [ ]:

lbl = preprocessing.LabelEncoder()

catData = ['dyn_ptal', 'dyn_flood_risk','crime_rating']
for att in catData:
    unique_classes = Data[att].unique()
    one_hot = pd.get_dummies(unique_classes, prefix=att,dummy_na=True, columns = [att])
    one_hot[att] = unique_classes
    Data = Data.merge(one_hot, on = [att], how='left')
    Data = Data.drop(columns = [att])

In [ ]:
list(Data.columns)

### Define sq_price as the model goal 

In [ ]:
Data['price'] = Data['dyn_sold_sqf_average']
Data.drop(['dyn_sold_sqf_average'], axis = 1, inplace = True)

In [ ]:
list(Data.columns)

In [ ]:
len(list(Data.columns))

In [ ]:
# Split data
X, y = Data.iloc[:, :-1], Data.iloc[:, -1]

In [ ]:
X= X.replace([np.inf, -np.inf], 0).fillna(0)

X.dropna(axis=0, how='any', inplace=True)
y = y.loc[X.index]

#PCA Decomposition

317 explanatory variables. Need to figure out if:
    
- I could speed up the training of regression algorithm by decomposing the original data with PCA

- if the principal components themselves could tell us something about the variance, and what the most important variables seem to be.

Decompose the data into 10 principal components

In [ ]:
def pca_dec(data, n):
  pca = PCA(n)
  X_dec = pca.fit_transform(data)
  return X_dec, pca

#Decomposing the train set:
pca_train_results, pca_train = pca_dec(X, 10)

#Decomposing the test set:
#pca_test_results, pca_test = pca_dec(X, 10)

#Creating a table with the explained variance ratio
names_pcas = [f"PCA Component {i}" for i in range(1, 11, 1)]
scree = pd.DataFrame(list(zip(names_pcas, pca_train.explained_variance_ratio_)), columns=["Component", "Explained Variance Ratio"])
print(scree)

In [ ]:
plt.plot(np.cumsum(pca_train.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

Note that only two components describe 99% of the data variance.

The 1º component describe 94% of the data variance

Each of the subcomponents of the 1º PCA corresponds to one of the 317 variables. Let's take the frist principal component and look into each subcomponent and rank them for their magnitude. 

In [ ]:
#Sorting the values of the first principal component by how large each one is
df = pd.DataFrame({'PCA':pca_train.components_[0], 'Variable Names':list(X.columns)})
df = df.sort_values('PCA', ascending=False)

df[:15]

2020_value the price growth on 2020 seems to be, the most positive important descritive factor in the 1º component. 

In [ ]:
df[-15:]

2020_value the price growth on 2020 seems to be, the most negative important descritive factor in the 1º component. 

Taking the 2º principal component, let's look at each subcomponent and rank them for their magnitude. 

In [ ]:
#Sorting the values of the first principal component by how large each one is
df = pd.DataFrame({'PCA':pca_train.components_[1], 'Variable Names':list(X.columns)})
df = df.sort_values('PCA', ascending=False)


df[:15]

In [ ]:
df[-10:]

Taking the 3º principal component, let's look at each subcomponent and rank them for their magnitude.

In [ ]:
#Sorting the values of the first principal component by how large each one is
df = pd.DataFrame({'PCA':pca_train.components_[2], 'Variable Names':list(X.columns)})
df = df.sort_values('PCA', ascending=False)

df[:15]

In [ ]:
df[-15:]

In [ ]:
# Preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
## SVR
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR, SVR

In [ ]:
# Splitting
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                      test_size = 0.2)

In [ ]:
stackingEnsemble = Pipeline([('StandardScaler', StandardScaler()),
                             ('PCA', PCA(n_components=40)),
                             ('SVR', SVR(kernel='rbf', C=1.4052512400014436, gamma=0.002198643705042812, max_iter=10000))])

stackingEnsemble.fit(train_X, train_y)

In [ ]:
pred_train = stackingEnsemble.predict(train_X)
pred_test = stackingEnsemble.predict(test_X)

In [ ]:
# MAE Computation
mae =MAE(train_y, pred_train)
print("Train MAE : % f --- %f" %(mae,mae/715.212067*100),"%")
# MAE Computation
mae =MAE(test_y, pred_test)
print("Test MAE : % f --- %f" %(mae,mae/715.212067*100),"%")

# MAE Computation
mape =MAPE(train_y, pred_train)
print("Train MAPE : % f --- %f" %(mape,mape/715.212067*100),"%")
# MAE Computation
mape =MAPE(test_y, pred_test)
print("Test MAPE : % f --- %f" %(mape,mape/715.212067*100),"%")

# MAE Computation
r2 =R2(train_y, pred_train)
print("Train R2 : % f" %(r2))
# MAE Computation
r2 =R2(test_y, pred_test)
print("Test R2 : % f" %(r2))

Train MAE :  0.819866 --- 0.114633 %
Test MAE :  15.438016 --- 2.158523 %
Train MAPE :  0.555237 --- 0.077632 %
Test MAPE :  10.851685 --- 1.517268 %
Train R2 :  0.999891
Test R2 :  0.938633